# Ingestion of COSMODC2 (Korytov et al.) for WeakLensingDeblending (WLD)

This notebook is intended to be used at NERSC or at an environment where `GCRCatalogs` points to the correct directories to retrieve the relevant data from cosmoDC2.

The purpose of this notebook is to illustrate how to query the cosmoDC2 catalog and output a WLD-compatible input catalog.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import GCRCatalogs # We'll query the catalogs using GCR

In [3]:
gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image') # This is the cosmoDC2 version that is used for DC2 run 2.x (goes up to z=3)

In [4]:
gc.list_all_quantities() # Let's see what quantities are included in the catalog

['Mag_true_z_sdss_z0',
 'sed_1246_306_no_host_extinction',
 'sed_5467_339_disk',
 'sed_1246_306_bulge',
 'sed_4848_300_disk',
 'sed_2407_591_disk_no_host_extinction',
 'Mag_true_u_sdss_z0',
 'sed_4565_283_disk',
 'sed_15143_2259',
 'sed_3381_209_disk',
 'sed_13177_1966_bulge',
 'sed_1552_381_bulge_no_host_extinction',
 'sed_6166_382_disk_no_host_extinction',
 'sed_6954_431_bulge_no_host_extinction',
 'sed_5806_360_bulge',
 'sed_15143_2259_bulge',
 'sed_4048_251_bulge',
 'sed_8329_517_disk',
 'mag_g',
 'mag_r_lsst',
 'sed_4048_251_disk_no_host_extinction',
 'sed_1552_381_no_host_extinction',
 'sed_6548_406_bulge_no_host_extinction',
 'sed_1933_474_bulge_no_host_extinction',
 'sed_1000_246',
 'sed_3590_222_disk_no_host_extinction',
 'R_v_disk',
 'sersic_disk',
 'mag_true_u_lsst',
 'sed_3812_236',
 'sed_3590_222',
 'sed_17402_2596_no_host_extinction',
 'ellipticity_2_bulge_true',
 'mag_true_y_lsst_no_host_extinction',
 'sed_6166_382_no_host_extinction',
 'sed_4048_251_no_host_extinction',

In [4]:
tab_out = dict() # Prepare a dictionary where we are going to dump the columns. This can be done more elegantly with GCR aliases but alas

In [5]:
# Perform the query. We select 24 sq-deg (quite large area). 
# For smaller queries reduce the ra, dec range and/or filter by healpixel numbers using `native_filters` (faster)
data = gc.get_quantities(['galaxy_id','ra','dec','mag_u_lsst','mag_g_lsst','mag_r_lsst',
                          'mag_i_lsst','mag_z_lsst','mag_y_lsst','redshift',
                          'size_bulge_true', 'size_disk_true',
                          'size_minor_disk_true', 'size_minor_bulge_true',
                         'bulge_to_total_ratio_i','position_angle_true',
                         'shear_1','shear_2'], filters=['ra >= 52 ', 'ra <= 58', 'dec >= -32', 'dec <= -28'])

In [6]:
len(data['ra']) #Count the number of objects

106579095

In [7]:
import fitsio #Get fitsio to dump the data in a FITS file

In [8]:
# Format conversion
tab_out['galtileid']=data['galaxy_id']
tab_out['ra'] = data['ra']
tab_out['dec'] = data['dec']
for band in ['u','g','r','i','z','y']:
    tab_out[f'{band}_ab'] = data[f'mag_{band}_lsst']
tab_out['a_b'] = data['size_bulge_true']
tab_out['b_b'] = data['size_minor_bulge_true']
tab_out['a_d'] = data['size_disk_true']
tab_out['b_d'] = data['size_minor_disk_true']
tab_out['pa_bulge'] = data['position_angle_true']
tab_out['pa_disk'] = tab_out['pa_bulge']
tab_out['fluxnorm_agn'] = np.zeros(len(data['ra']))
tab_out['redshift'] = data['redshift']
tab_out['fluxnorm_bulge'] = data['bulge_to_total_ratio_i']
tab_out['fluxnorm_disk'] = 1-tab_out['fluxnorm_bulge']

In [9]:
#Write to SCRATCH
fitsio.write('$SCRATCH/cosmoDC2_v1.1.4_image_WLD_ra_52_58_dec_-32_-28_all_unsheared.fits', tab_out)